# 02 — Model sweep

Compare a few different models on the same dataset. This helps validate that Cleanlab findings are not model-specific.

In [1]:
from pathlib import Path
import sys

cwd = Path.cwd()
if (cwd / "src").exists():
    sys.path.insert(0, str(cwd / "src"))
elif (cwd.parent / "src").exists():
    sys.path.insert(0, str(cwd.parent / "src"))


In [2]:
import pandas as pd

from cleanlab_demo.config import CleanlabConfig, DatasetName, DemoConfig, ModelName, RunConfig
from cleanlab_demo.experiments import run_sweep

base = RunConfig(
    dataset=DatasetName.adult_income,
    cleanlab=CleanlabConfig(enabled=False),
    demo=DemoConfig(max_rows=6000),
)

rows = run_sweep(
    dataset=DatasetName.adult_income,
    models=[ModelName.logistic_regression, ModelName.hist_gradient_boosting, ModelName.random_forest],
    base_config=base,
)

df = pd.DataFrame([r.model_dump() for r in rows]).sort_values("primary_metric", ascending=False)
df


2026-02-07 21:30:59 | INFO     | cleanlab_demo | Loading dataset: adult_income
2026-02-07 21:30:59 | INFO     | cleanlab_demo | Dataset loaded: 6,000 rows
2026-02-07 21:30:59 | INFO     | cleanlab_demo | Loading dataset: adult_income
2026-02-07 21:30:59 | INFO     | cleanlab_demo | Dataset loaded: 6,000 rows
2026-02-07 21:31:00 | INFO     | cleanlab_demo | Loading dataset: adult_income
2026-02-07 21:31:00 | INFO     | cleanlab_demo | Dataset loaded: 6,000 rows


,dataset,task,model,primary_metric,metrics,n_label_issues
1,DatasetName.adult_income,TaskType.classification,ModelName.hist_gradient_boosting,0.923760,"{'accuracy': 0.8716666666666667, 'f1_weighted'...",0
2,DatasetName.adult_income,TaskType.classification,ModelName.random_forest,0.910169,"{'accuracy': 0.8533333333333334, 'f1_weighted'...",0
0,DatasetName.adult_income,TaskType.classification,ModelName.logistic_regression,0.908543,"{'accuracy': 0.8633333333333333, 'f1_weighted'...",0
